In [25]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/temporal-tagging-and-ordering/test.json
/kaggle/input/temporal-tagging-and-ordering/dataset.csv


In [26]:
import spacy
import json

In [27]:
! pip install datasets
! pip install transformers[torch]
! pip install tokenizers
! pip install evaluate
! pip install rouge_score
! pip install sentencepiece
! pip install huggingface_hub

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [28]:
!pip install timexy

In [29]:
import nltk
import evaluate
from datasets import load_dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [30]:
from datasets import Dataset, DatasetDict

In [31]:
import timexy
from timexy import Timexy
from timexy import rule
from timexy.languages import en

In [32]:
nlp = spacy.load("en_core_web_sm")

# Optionally add config if varying from default values
config = {
    "kb_id_type": "timex3",  # possible values: 'timex3'(default), 'timestamp'
    "label": "timexy",       # default: 'timexy'
    "overwrite": False       # default: False
}
nlp.add_pipe("timexy", config=config, before="ner")

In [33]:
df = pd.read_csv('/kaggle/input/temporal-tagging-and-ordering/dataset.csv')

In [34]:
# train (1081)
c=0
for index, row in df.iterrows():
    if(row['Chunk']<=1081):
        c+=1
train = df.iloc[:c,:]
test = df.iloc[c:,:]
# validate (rest)

In [35]:
# Run it the first time
# MODEL_NAME = "google/flan-t5-base"

# tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
# model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

chkpnt = './input/checkpoints/chkpnt1'
model.save_pretrained(chkpnt)
tokenizer.save_pretrained(chkpnt)

('./input/checkpoints/chkpnt1/tokenizer_config.json',
 './input/checkpoints/chkpnt1/special_tokens_map.json',
 './input/checkpoints/chkpnt1/spiece.model',
 './input/checkpoints/chkpnt1/added_tokens.json')

In [36]:
model = T5ForConditionalGeneration.from_pretrained(chkpnt)
tokenizer = T5Tokenizer.from_pretrained(chkpnt)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [37]:
import re
def preprocess(batch):
    p,clean_batch = [],[]
    for i in batch['Inputs']:
        for j in i.split('\n')[:-1]:
            x = re.split('~__[True|False]__',j)
            print(x)
            clean_batch.append(x)
            if(x[1][3:7]=='True'):
                p.append(True)
            else:
                p.append(False)
    inputs = ["Generate : " + sentence for sentence in clean_batch]
    
    model_inputs = tokenizer(inputs,truncation=True)
    labels = tokenizer(text_target=batch['Outputs'],truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs,p

In [38]:
tokenized_dataset,p = preprocess(train)

IndexError: list index out of range

In [13]:
# Training Arguments
L_RATE = 3e-4
BATCH_SIZE = 4
PER_DEVICE_EVAL_BATCH = 2
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIM = 3
NUM_EPOCHS = 3

# Set up training arguments
training_args = Seq2SeqTrainingArguments(
   output_dir="./results",
   evaluation_strategy="epoch",
   learning_rate=L_RATE,
   per_device_train_batch_size=BATCH_SIZE,
   per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
   weight_decay=WEIGHT_DECAY,
   save_total_limit=SAVE_TOTAL_LIM,
   num_train_epochs=NUM_EPOCHS,
   predict_with_generate=True,
   push_to_hub=False
)

In [14]:
trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_dataset["train"],
   eval_dataset=tokenized_dataset["test"],
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics
)

NameError: name 'tokenized_dataset' is not defined